Problem Statement.

You are given an integer n indicating the number of people in a network. Each person is labeled from 0 to n - 1.

You are also given a 0-indexed 2D integer array restrictions, where restrictions[i] = [xi, yi] means that person xi and person yi cannot become friends, either directly or indirectly through other people.

Initially, no one is friends with each other. You are given a list of friend requests as a 0-indexed 2D integer array requests, where requests[j] = [uj, vj] is a friend request between person uj and person vj.

A friend request is successful if uj and vj can be friends. Each friend request is processed in the given order (i.e., requests[j] occurs before requests[j + 1]), and upon a successful request, uj and vj become direct friends for all future friend requests.

Return a boolean array result, where each result[j] is true if the jth friend request is successful or false if it is not.

Note: If uj and vj are already direct friends, the request is still successful.

 

Example 1:

Input: n = 3, restrictions = [[0,1]], requests = [[0,2],[2,1]]
Output: [true,false]
Explanation:
Request 0: Person 0 and person 2 can be friends, so they become direct friends. 
Request 1: Person 2 and person 1 cannot be friends since person 0 and person 1 would be indirect friends (1--2--0).

Example 2:

Input: n = 3, restrictions = [[0,1]], requests = [[1,2],[0,2]]
Output: [true,false]
Explanation:
Request 0: Person 1 and person 2 can be friends, so they become direct friends.
Request 1: Person 0 and person 2 cannot be friends since person 0 and person 1 would be indirect friends (0--2--1).

Example 3:

Input: n = 5, restrictions = [[0,1],[1,2],[2,3]], requests = [[0,4],[1,2],[3,1],[3,4]]
Output: [true,false,true,false]
Explanation:
Request 0: Person 0 and person 4 can be friends, so they become direct friends.
Request 1: Person 1 and person 2 cannot be friends since they are directly restricted.
Request 2: Person 3 and person 1 can be friends, so they become direct friends.
Request 3: Person 3 and person 4 cannot be friends since person 0 and person 1 would be indirect friends (0--4--3--1).

Constraints:

    2 <= n <= 1000
    0 <= restrictions.length <= 1000
    restrictions[i].length == 2
    0 <= xi, yi <= n - 1
    xi != yi
    1 <= requests.length <= 1000
    requests[j].length == 2
    0 <= uj, vj <= n - 1
    uj != vj

# Union Find - O(REQ * RES) runtime, O(N) space

In [2]:
from typing import List

class Solution:
    def friendRequests(self, n: int, restrictions: List[List[int]], requests: List[List[int]]) -> List[bool]:
        uf = {i: i for i in range(n)}
        
        def find(i):
            if i != uf[i]:
                uf[i] = find(uf[i])
                
            return uf[i]

        res = []
        for i, j in requests:
            success = True
            pi, pj = find(i), find(j)
            if pi != pj:
                for x, y in restrictions:
                    px, py = find(x), find(y)
                    if (px, py) == (pi, pj) or (px, py) == (pj, pi):
                        success = False
                        break
                    
            if success: uf[pj] = pi
            res.append(success)
            
        return res

# Optimized Union Find - O(REQ * RES) runtime, O(N) spac

In [5]:
from typing import List
from collections import defaultdict

class Solution:
    def friendRequests(self, n: int, restrictions: List[List[int]], requests: List[List[int]]) -> List[bool]:
        parents = [i for i in range(n)]
        ranks = [0] * n
        forbidden = defaultdict(set)
        for i, j in restrictions:
            forbidden[i].add(j)
            forbidden[j].add(i)
        
        def find(i):
            if i != parents[i]:
                parents[i] = find(parents[i])
            return parents[i]
        
        def union(p1, p2):
            if ranks[p1] > ranks[p2]:
                parents[p2] = p1
            elif ranks[p1] < ranks[p2]:
                parents[p1] = p2
                p1, p2 = p2, p1
            else:
                ranks[p1] += 1
                parents[p2] = p1
                
            forbidden[p1] |= forbidden[p2]
            for i in forbidden[p2]:
                forbidden[i].remove(p2)
                forbidden[i].add(p1)
            del forbidden[p2]
        
        ans = []
        for i, j in requests:
            p1 = find(i)
            p2 = find(j)
            if p1 == p2:
                ans.append(True)         
            elif p2 in forbidden[p1]:
                ans.append(False)
            else:
                union(p1, p2)
                ans.append(True)

        return ans

In [6]:
instance = Solution()
instance.friendRequests(3, [[0,1]], [[0,2],[2,1]])

[True, False]